four points to work on :-
1. rotating user agent's and other saving mechanisms
2. url join
3. is joined url is valid 
4. store set of asin ids
5. appropriate waits


In [1]:
from selenium import webdriver
from pathlib import Path
import requests
import time
import random
from itertools import repeat
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from aryaman.scripts.category.category_enums import *
import pandas as pd
from concurrent.futures import ThreadPoolExecutor


pandas dataframe to contain main category , tag , asin id and created product links

In [2]:
products = pd.DataFrame(columns=['Main Category','Tag','ASIN','Product Link'])
asins_set = set()

functions

In [44]:
def create_url(asin_id):
    return 'https://www.amazon.in/dp/{asin_id}'.format(asin_id=asin_id)

def check_link_validity(url,session=None):
    time.sleep(random.uniform(1,7))
    if session == None:
        response = requests.get(url=url,allow_redirects=True,timeout=200)
    else:
        response = session.get(url=url, allow_redirects=True, timeout=200)
    # head() was giving 405 code because amazon server expects get() on those links , but get() is costly therefore
    # multithreading
    # print(response.status_code)
    try:
        if response.status_code==200 :
            if ('/dp/' in response.url or '/gp/product/' in response.url):
                return True
            else:
                print(f"Redirected to a non-product page: {response.url}")
                return False
        else:
            print("status code not 200:",response.url)
            return False
    except Exception as e:
        print(f"Exception occurred in checking link validity : {e}")
def check_asin_duplicate(set_of_asins,asin_id):
    if asin_id in set_of_asins:
        return True
    else:
        return False
    

def update_products_dataframe_and_asins_set(products_dataframe,set_of_asins,new_asin_ids_list,main_category,tag):
    urls = []
    chosen_ids = []
    for id in new_asin_ids_list:
        if not check_asin_duplicate(set_of_asins,id):
            set_of_asins.add(id)
            urls.append(create_url(id))
            chosen_ids.append(id)
    with requests.Session() as session:
        session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive'
        })
        with ThreadPoolExecutor(max_workers=len(urls)) as executor:
            responses = list(executor.map(check_link_validity,urls,repeat(session)))
    
    for i in range(len(responses)):
        if responses[i] == True:
            new_row = {'Main Category':main_category,'Tag':tag,'ASIN':chosen_ids[i],'Product Link':urls[i]}
            products_dataframe = pd.concat([products_dataframe,pd.DataFrame([new_row])],ignore_index=True)
            products_dataframe.reset_index(inplace=True,drop=True)
            # print(products_dataframe)
    return products_dataframe

def scrape_asin_ids(driver,css_selector_string):
    main_slot_div = driver.find_elements(By.CSS_SELECTOR,"div.s-main-slot.s-result-list")
    if len(main_slot_div) > 1:
        raise Exception("more than one main slots identified")
    asin_elems = main_slot_div[0].find_elements(By.CSS_SELECTOR,css_selector_string)
    asin_ids = []
    for elem in asin_elems:
        val = elem.get_attribute("data-asin")
        if isinstance(val,str) and len(val) == 10 and val.isalnum():
            asin_ids.append(val)
    return asin_ids


global variables

In [4]:
# set them inside the code
current_main_category = None 
current_tag_of_main_category = None

driver + scraping

In [53]:
driver = None
def load_driver():
    global driver
    webdriver_path = "..\chromedriver.exe"
    cService = webdriver.ChromeService(executable_path=webdriver_path)
    chrome_options = Options()
    chrome_options.add_experimental_option("detach",True)
    driver = webdriver.Chrome(service=cService,options=chrome_options)
    driver.get("https://www.amazon.in/")
    # driver.maximize_window()

# Baby Products

In [6]:
current_main_category = BabyProducts.get_class_name()

## Diapers

In [8]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Diapers)

In [11]:
products = update_products_dataframe_and_asins_set(products,asins_set,scrape_asin_ids(driver,"div[data-asin]"),current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)

## Baby Wipes

In [7]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Baby_Wipes)

In [25]:
products = update_products_dataframe_and_asins_set(products,asins_set,scrape_asin_ids(driver,"div[data-asin]"),current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)

## Baby Clothing

In [26]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Baby_Clothing)
products = update_products_dataframe_and_asins_set(products,asins_set,scrape_asin_ids(driver,"div[data-asin]"),current_main_category,current_tag_of_main_category)

In [28]:
products.to_csv('products.csv',index=None)

## Strollers

In [37]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Strollers)
products = update_products_dataframe_and_asins_set(products,asins_set,scrape_asin_ids(driver,"div[data-asin]"),current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)

In [38]:
products[products['Tag']=='Strollers'].shape

(75, 4)

## Car seats

In [40]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Car_Seats)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(scraped_ids))

67


In [41]:
products.tail(4)

,Main Category,Tag,ASIN,Product Link
310,Baby Products,Car Seats,B0B8SS1XKK,https://www.amazon.in/dp/B0B8SS1XKK
311,Baby Products,Car Seats,B0DCZPHR2G,https://www.amazon.in/dp/B0DCZPHR2G
312,Baby Products,Car Seats,B0D4QV9RH4,https://www.amazon.in/dp/B0D4QV9RH4
313,Baby Products,Car Seats,B07MZHSVC7,https://www.amazon.in/dp/B07MZHSVC7


## Baby Toys

In [42]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Baby_Toys)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(scraped_ids))

71


In [43]:
products.tail(10)

,Main Category,Tag,ASIN,Product Link
371,Baby Products,Baby Toys,B0D6Z1YPTZ,https://www.amazon.in/dp/B0D6Z1YPTZ
372,Baby Products,Baby Toys,B0BKZMHP48,https://www.amazon.in/dp/B0BKZMHP48
373,Baby Products,Baby Toys,B0D3H436PW,https://www.amazon.in/dp/B0D3H436PW
374,Baby Products,Baby Toys,B08R9QVNYX,https://www.amazon.in/dp/B08R9QVNYX
375,Baby Products,Baby Toys,B074Y7KQD1,https://www.amazon.in/dp/B074Y7KQD1
376,Baby Products,Baby Toys,B0D49SCYDC,https://www.amazon.in/dp/B0D49SCYDC
377,Baby Products,Baby Toys,B0CVQMCW9J,https://www.amazon.in/dp/B0CVQMCW9J
378,Baby Products,Baby Toys,B09VDCVQH4,https://www.amazon.in/dp/B09VDCVQH4
379,Baby Products,Baby Toys,B083F2X9YS,https://www.amazon.in/dp/B083F2X9YS
380,Baby Products,Baby Toys,B0D5CY745L,https://www.amazon.in/dp/B0D5CY745L


## Baby Bathing products

In [45]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Baby_Bathing_Products)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(scraped_ids))

71


## Baby Skin Care products

In [47]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Baby_Skincare_Products)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(scraped_ids))

68


## Baby Feeding bottles


In [65]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Baby_Feeding_Bottles)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(scraped_ids))

52


In [66]:
products[products['Tag']=='Baby Feeding Bottles'].shape

(100, 4)

## Nursing and BreastFeeding Products

In [67]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Nursing_and_Breastfeeding_Products)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(scraped_ids))

61


## Cribs And Bassinets

In [68]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Cribs_and_Bassinets)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(scraped_ids))

60


## Baby Carriers

In [69]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Baby_Carriers)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(scraped_ids))

73


# Beauty and Personal Care

In [2]:
current_main_category = BeautyAndPersonalCare.get_class_name()

NameError: name 'BeautyAndPersonalCare' is not defined

## Skin Care Products

In [3]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Skin_Care_Products)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(scraped_ids))

NameError: name 'BeautyAndPersonalCare' is not defined

## Hair care products

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Hair_Care_Products)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

72


## Make up kits

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Makeup_Kits)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

58


In [ ]:
products.tail(3)

,Main Category,Tag,ASIN,Product Link
950,Beauty & Personal Care,Makeup Kits,B0DKN5825L,https://www.amazon.in/dp/B0DKN5825L
951,Beauty & Personal Care,Makeup Kits,B0DKJLHPK8,https://www.amazon.in/dp/B0DKJLHPK8
952,Beauty & Personal Care,Makeup Kits,B0CSTCW4T7,https://www.amazon.in/dp/B0CSTCW4T7


## Perfumes

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Perfumes)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

61


## Shampoos and Conditioners

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Shampoos_and_Conditioners)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

57


## Hair Styling Tools

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Hair_Styling_Tools)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

54


## Oral Care Products

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Oral_Care_Products)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

59


## Bath and body wash

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Bath_and_Body_Wash)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

60


## Mens grooming kits

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Mens_Grooming_Kits)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

116


## Face creams and serums

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Face_Creams_and_Serums)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

61


## Body Lotions and Moisturizers

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Body_Lotions_and_Moisturizers)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

105


## Shaving and hair removal products

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Shaving_and_Hair_Removal_Products)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

65


## Nail Polish and nail care

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Nail_Polish_and_Nail_Care)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

68


## essential oils

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Essential_Oils)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

61


## sunscreens and tanning products

In [ ]:
current_tag_of_main_category = BeautyAndPersonalCare.get_sub_name(BeautyAndPersonalCare.Sunscreens_and_Tanning_Products)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

58


# Books

In [105]:
current_main_category = Books.get_class_name()

## Fiction Books

In [107]:
current_tag_of_main_category = Books.get_sub_name(Books.Fiction_Books)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

57


## Non Fiction Books

In [110]:
current_tag_of_main_category = Books.get_sub_name(Books.Non_Fiction_Books)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

69


## childrens story books

In [112]:
current_tag_of_main_category = Books.get_sub_name(Books.Childrens_Story_Books)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

61


## Self Help books

In [116]:
current_tag_of_main_category = Books.get_sub_name(Books.Self_Help_Books)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

67


## Academic Textbooks

In [122]:
current_tag_of_main_category = Books.get_sub_name(Books.Academic_Textbooks)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

73


## biographies and auto-biographies

In [124]:
current_tag_of_main_category = Books.get_sub_name(Books.Biographies_and_Autobiographies)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

54


## cookbooks and recipe books

In [126]:
current_tag_of_main_category = Books.get_sub_name(Books.Cookbooks_and_Recipe_Books)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

58


## mystery and thriller books

In [129]:
current_tag_of_main_category = Books.get_sub_name(Books.Mystery_and_Thriller_Books)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

58


## Science fiction books

In [132]:
current_tag_of_main_category = Books.get_sub_name(Books.Science_Fiction_Books)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

62


## Romance Novels

In [135]:
current_tag_of_main_category = Books.get_sub_name(Books.Romance_Novels)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

64


## Poetry Books

In [138]:
current_tag_of_main_category = Books.get_sub_name(Books.Poetry_Books)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

66


# Clothing Shoes and Jewelry

In [139]:
current_main_category = ClothingShoesAndJewelry.get_class_name()

## Mens T shirts

In [142]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Mens_T_Shirts)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

205


## Womens Dresses

In [145]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Womens_Dresses)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

197


## sneakers for men

In [147]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Sneakers_for_Men)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

138


## womens heels

In [151]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Womens_Heels)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

192


## kids clothing sets

In [153]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Kids_Clothing_Sets)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

122


## womens jewelry

In [156]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Womens_Jewelry)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

196


## Mens watches

In [158]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Mens_Watches)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

130


## Casual shoes for men

In [160]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Casual_Shoes_for_Men)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

116


## sports shoes for women

In [162]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Sports_Shoes_for_Women)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

132


## women handbags

In [164]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Womens_Handbags)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

135


## men's belts and wallets

In [166]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Mens_Belts_and_Wallets)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

119


## women's scarves and wraps

In [168]:
current_tag_of_main_category = ClothingShoesAndJewelry.get_sub_name(ClothingShoesAndJewelry.Womens_Scarves_and_Wraps)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

132


# Electronics

In [169]:
current_main_category = Electronics.get_class_name()

## Mobile Phones

In [185]:
current_tag_of_main_category = Electronics.get_sub_name(Electronics.Mobile_Phones)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

254


## Laptops

In [197]:
current_tag_of_main_category = Electronics.get_sub_name(Electronics.Laptops)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

209


## earphones and headphones

In [206]:
current_tag_of_main_category = Electronics.get_sub_name(Electronics.Earphones_and_Headphones)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

207


## Smartwatches

In [210]:
current_tag_of_main_category = Electronics.get_sub_name(Electronics.Smartwatches)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

95


## Power Banks

In [213]:
current_tag_of_main_category = Electronics.get_sub_name(Electronics.Power_Banks)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

70


## Tablets

In [217]:
current_tag_of_main_category = Electronics.get_sub_name(Electronics.Tablets)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

78


## Bluetooth Speakers

In [220]:
current_tag_of_main_category = Electronics.get_sub_name(Electronics.Bluetooth_Speakers)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

84


## Mobile Accessories

In [223]:
current_tag_of_main_category = Electronics.get_sub_name(Electronics.Mobile_Accessories)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

79


## Smart TVs

In [226]:
current_tag_of_main_category = Electronics.get_sub_name(Electronics.Smart_TVs)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

59


## Gaming consoles

In [227]:
current_tag_of_main_category = Electronics.get_sub_name(Electronics.Gaming_Consoles)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

57


## Printers and scanners

In [232]:
current_tag_of_main_category = Electronics.get_sub_name(Electronics.Printers_and_Scanners)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

98


# Home and Kitchen

In [233]:
current_main_category = HomeAndKitchen.get_class_name()

## Kitchen appliances

In [236]:
current_tag_of_main_category = HomeAndKitchen.get_sub_name(HomeAndKitchen.Kitchen_Appliances)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

59


## cookware sets

In [237]:
current_tag_of_main_category = HomeAndKitchen.get_sub_name(HomeAndKitchen.Cookware_Sets)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

56


## bed sheets and linens

In [238]:
current_tag_of_main_category = HomeAndKitchen.get_sub_name(HomeAndKitchen.Bed_Sheets_and_Linens)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

61


## Home decor items

In [239]:
current_tag_of_main_category = HomeAndKitchen.get_sub_name(HomeAndKitchen.Home_Decor_Items)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

69


## storage solutions


In [240]:
current_tag_of_main_category = HomeAndKitchen.get_sub_name(HomeAndKitchen.Storage_Solutions)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

66


## cleaning supplies

In [241]:
current_tag_of_main_category = HomeAndKitchen.get_sub_name(HomeAndKitchen.Cleaning_Supplies)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

61


## dining and serveware

In [242]:
current_tag_of_main_category = HomeAndKitchen.get_sub_name(HomeAndKitchen.Dining_and_Serveware)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

65


## water purifiers

In [244]:
current_tag_of_main_category = HomeAndKitchen.get_sub_name(HomeAndKitchen.Water_Purifiers)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

46


## curtains and drapes

In [245]:
current_tag_of_main_category = HomeAndKitchen.get_sub_name(HomeAndKitchen.Curtains_and_Drapes)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

71


## bath towels and mats

In [246]:
current_tag_of_main_category = HomeAndKitchen.get_sub_name(HomeAndKitchen.Bath_Towels_and_Mats)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

68


## home furniture sets

In [247]:
current_tag_of_main_category = HomeAndKitchen.get_sub_name(HomeAndKitchen.Home_Furniture_Sets)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

62


# sports and outdoors

In [248]:
current_main_category = SportsAndOutdoors.get_class_name()

## fitness equipments

In [249]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Fitness_Equipment)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

67


## running shoes

In [250]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Running_Shoes)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

65


## yoga mats

In [251]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Yoga_Mats)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

63


## sportswear for men

In [252]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Sportswear_for_Men)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

65


## sportswear women

In [253]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Sportswear_for_Women)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

65


## bicycle and cycle accessories

In [255]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Bicycles_and_Cycling_Accessories)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

92


## swimming gear

In [256]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Swimming_Gear)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

60


## cricket bats and balls

In [257]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Cricket_Bats_and_Balls)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

61


## badminton 

In [258]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Badminton_Racquets_and_Shuttlecocks)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

60


## football

In [260]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Footballs_and_Accessories)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

126


## gym

In [261]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Gym_Gloves_and_Accessories)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

55


## water bottles and shakers

In [262]:
current_tag_of_main_category = SportsAndOutdoors.get_sub_name(SportsAndOutdoors.Water_Bottles_and_Shakers)
scraped_ids = scrape_asin_ids(driver,"div[data-asin]")
products = update_products_dataframe_and_asins_set(products,asins_set,scraped_ids,current_main_category,current_tag_of_main_category)
products.to_csv('products.csv',index=None)
print(len(products[products['Tag']==current_tag_of_main_category]))

58


# further analysis

In [266]:
df = pd.read_csv('products.csv')

In [267]:
df.equals(products)

True